# Простая мнемоника

In [1]:
import pandas as pd
from numba import njit, jit
import hashlib, binascii
import os, time, random
from numba import njit, jit

In [2]:
from bip_utils import (
    Bip39MnemonicGenerator, Bip39MnemonicDecoder, Bip39SeedGenerator,
    Bip39WordsNum, Bip39Languages,
    Bip44, Bip44Coins, Bip44Changes,
    Bip49, Bip49Coins,
    Bip84, Bip84Coins,
    Bip86, Bip86Coins,
    Bip32Slip10Secp256k1, EthAddrEncoder
)

In [5]:
BTC_PATH = os.path.join('data','btc.txt'); 
ENG_PATH = os.path.join('data', 'english.txt');

In [7]:
add_set = pd.read_csv(BTC_PATH, sep = " ", header = None)[0];
words = pd.read_csv(ENG_PATH, sep = " ", header = None)[0];

In [10]:
# Консольные сообщения        
# getmsg();
# getmsg(list(add_set));

In [11]:
# TODO: Убрать лишние переменные (!)
# with open(ENG_PATH, 'r') as f:
#     i = 0;
#     for word in f:
#         nums[word.strip()] = i;
#         wordlist.append(word.strip());
#         i += 1;

Добавить запись ключа в файл

In [12]:
@njit(parallel = True)
def crypt(add_set, words, nums):
    # while True:
    # все что ниже крутится в цикле
    
    wordlist = ' '.join(random.sample(list(words), 11)).split();
    
    if len(wordlist) == 11:
        for i in range(2048):
            cand = wordlist + [words[i]];
            result = ' '.join(cand);
            if isOk(cand, nums):
                result, seed44, seed49, seed84, seed86 = seedgenerator(wordlist);
    %%timeit

Убрать переменную `nums`

In [13]:
def isOk(ws, nums):
    N = 0;
    for w in ws:
        # TODO: Переделать получение значения по "слову-ключу"
        N = (N << 11) + nums.get(w.lower(), 0);
    nhex = format(N, '033x');
    h = hashlib.sha256(binascii.unhexlify(nhex[:-1])).hexdigest();
    return h[0] == nhex[-1];

In [71]:
def seedgenerator(ddd):
    seed_bytes = Bip39SeedGenerator(result).Generate();
    # TODO: threadin 1
    bip44_mst_ctx = Bip44.FromSeed(seed_bytes, Bip44Coins.BITCOIN);
    bip44_acc_ctx = bip44_mst_ctx.Purpose().Coin().Account(0);
    bip44_chg_ctx = bip44_acc_ctx.Change(Bip44Changes.CHAIN_EXT);
    # TODO: threadin 2
    bip49_mst_ctx = Bip49.FromSeed(seed_bytes, Bip49Coins.BITCOIN);
    bip49_acc_ctx = bip49_mst_ctx.Purpose().Coin().Account(0);
    bip49_chg_ctx = bip49_acc_ctx.Change(Bip44Changes.CHAIN_EXT);
    # TODO: threadin 3
    bip84_mst_ctx = Bip84.FromSeed(seed_bytes, Bip84Coins.BITCOIN);
    bip84_acc_ctx = bip84_mst_ctx.Purpose().Coin().Account(0);
    bip84_chg_ctx = bip84_acc_ctx.Change(Bip44Changes.CHAIN_EXT);
    # TODO: threadin 4
    bip86_mst_ctx = Bip86.FromSeed(seed_bytes, Bip86Coins.BITCOIN);
    bip86_acc_ctx = bip86_mst_ctx.Purpose().Coin().Account(0);
    bip86_chg_ctx = bip86_acc_ctx.Change(Bip44Changes.CHAIN_EXT);
    print(result);
    for i in range(20):
        # TODO: bip32_der_ctx = bip32_mst_eth.DerivePath("m/44'/60'/0'/0/" + str(i))

        # TODO: threadin 1
        bip44_addr_ctx = bip44_chg_ctx.AddressIndex(i);
        seed44 = bip44_addr_ctx.PublicKey().ToAddress();
        # TODO: threadin 2
        bip49_addr_ctx = bip49_chg_ctx.AddressIndex(i);
        seed49 = bip49_addr_ctx.PublicKey().ToAddress();
        # TODO: threadin 3
        bip84_addr_ctx = bip84_chg_ctx.AddressIndex(i);
        seed84 = bip84_addr_ctx.PublicKey().ToAddress();
        # TODO: threadin 4
        bip86_addr_ctx = bip86_chg_ctx.AddressIndex(i);
        seed86 = bip86_addr_ctx.PublicKey().ToAddress();

        # TODO: eth_addr = EthAddrEncoder.EncodeKey(bip32_der_ctx.PublicKey().KeyObject())
        # TODO: eth = ((eth_addr) [2:]) # если ваши адреса ETH начинаются без 0x в начале

        # TODO: Раскоментировать стоку ниже, если ваши адреса ETH начинаются с 0x
        # TODO: eth = ((eth_addr)) # если ваши адреса ETH начинаются c 0x в начале

        # TODO: ПЕРЕДЕЛАТЬ НА PANDAS
        if seed44 in add_set or seed49 in add_set or seed84 in add_set or seed86 in add_set:
            print(result);
            return result, seed44, seed49, seed84, seed86;
            # d = open(f"seed.txt","a");
            # d.write(str(result) + '\n' + str(seed44) + '\n'+ str(seed49) + '\n'+ str(seed84) + '\n'+ str(seed86) + '\n');
            # d.flush();
            # d.close();
        elif len(ddd) == 12: print('OK' if isOk(ddd, nums) else 'Invalid', 'wordlist checksum');